In [ ]:
from multitcn_components import TCNStack, DownsampleLayerWithAttention, LearningRateLogger
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint, CSVLogger
from sklearn import preprocessing
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow_addons as tfa
import uuid
import sys

In [ ]:
### Set experiment seed
print("Enter a seed for the experiment:")
seed = input()
if len(seed)!=0 and seed.isdigit():
    seed = int(seed)
else:
    seed = 192
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
def log_experiment_details(just_print=False, complete=False):
    """ Log experiment details """
    if just_print:
        o = sys.stdout
    else:
        o = open(date_time_string+"--"+experiment_id+".txt","w")
    o.write("Date and time: %s \n" % date_time_string)
    o.write("Experiment complete : %s\n" % str(experiment_complete))
    if complete:
        now = datetime.now()
        completion_time = now.strftime("%d-%m-%Y-%H-%M-%S")
        o.write("Experiment complete at: %s\n" % completion_time)
        o.write("Duration in seconds: %f \n"% duration)
    o.write("Filename: %s\n" % str(sys.argv[0]))
    o.write("\n\n")
    o.write(" Training parameters ".center(100,"="))
    o.write("\n\n")
    o.write("Batch size: %d\n" % batch_size)
    o.write("Epochs : %d\n" % epochs)
    o.write("Device used: %s\n" % device)
    o.write("Random seed used: %s\n" % seed)
    o.write("Loss : %s\n" % loss)
    o.write("Optimizer config: %s" % str(optimizer.get_config()))
    o.write("\n\n")
    o.write(" Dataset parameters ".center(100,"="))
    o.write("\n\n")
    o.write("Dataset description: %s\n" % dataset_description)
    o.write("Number of input time series: %d\n" % num_input_time_series)
    o.write("Window length: %d\n" % window_length)
    o.write("Total sample size: %d \n"% total_samples)
    o.write("Training samples: %d\n" % int(train_x.shape[0]*training_percentage))
    o.write("Training start date: %s\n" % training_start_date)
    o.write("Validation samples: %d\n" % int(train_x.shape[0]*(1-training_percentage)))
    o.write("Test samples: %d\n" % test_x.shape[0])
    o.write("Test start date: %s\n" % holdout_set_start_date)
    o.write("Test end date: %s\n" % holdout_set_end_date)
    o.write("Experiment target: %s\n" % experiment_target)
    o.write("Dataset preprocessing: %s\n" % dataset_preprocessing)
    o.write("Shuffled training and val set: %s\n" % shuffle_train_set)
    o.write("Scaled output: %s\n" % scale_output)
    o.write("Input preprocessor details: %s, %s\n" %(preprocessor.__class__, preprocessor.get_params()))
    o.write("Output scaler details: %s, %s\n" %(out_preprocessor.__class__, out_preprocessor.get_params()))
    o.write("\n\n")
    o.write(" Specific model parameters ".center(100,"="))
    o.write("\n\n")
    o.write("tcn_kernel_size : %d\n" % tcn_kernel_size)
    o.write("tcn_filter_num : %d\n" % tcn_filter_num)
    o.write("tcn_layer_num : %d\n" % tcn_layer_num)
    o.write("tcn_use_bias : %s\n" % str(tcn_use_bias))
    o.write("tcn_kernel_initializer : %s\n" % tcn_kernel_initializer)
    o.write("tcn_dropout_rate : %0.2f\n" % tcn_dropout_rate)
    o.write("tcn_dropout_format : %s\n" % tcn_dropout_format)
    o.write("tcn_activation : %s\n" % tcn_activation)
    o.write("tcn_final_activation : %s\n" % tcn_final_activation)
    o.write("tcn_final_stack_activation : %s\n" % tcn_final_stack_activation)
    o.write("\n\n")
    o.write(" Additional useful notes ".center(100,"="))
    o.write("\n\n")
    o.write(additional_experiment_notes)
    o.write("\n\n")
    
    if not just_print:
        o.close()
    else:
        o.flush()

def windowed_dataset(series, time_series_number, window_size):
    """
    Returns a windowed dataset from a Pandas dataframe
    """
    available_examples= series.shape[0]-window_size + 1
    time_series_number = series.shape[1]
    inputs = np.zeros((available_examples,window_size,time_series_number))
    for i in range(available_examples):
        inputs[i,:,:] = series[i:i+window_size,:]
    return inputs 

def windowed_forecast(series, forecast_horizon):
    available_outputs = series.shape[0]- forecast_horizon + 1
    output_series_num = series.shape[1]
    output = np.zeros((available_outputs,forecast_horizon, output_series_num))
    for i in range(available_outputs):
        output[i,:]= series[i:i+forecast_horizon,:]
    return output

def shuffle_arrays_together(a,b):
    p = np.random.permutation(a.shape[0])
    return a[p],b[p]

def remove_outliers_and_interpolate(dataframe, std_times = 3):
    """
    Removes outliers further than std_times standard deviations from the mean of each column of a df and replaces them with simple interpolated values
    """
    for c in ['Temp_degC']:
        mask = (dataframe>40)
        dataframe.loc[mask[c],c] = np.nan

    for c in ['Turbidity_NTU','Chloraphylla_ugL']:
        mask = (dataframe<0)
        dataframe.loc[mask[c],c] = np.nan

    for c in list(dataframe.columns):
        mean = np.mean(np.array(dataframe[c]))
        std = np.std(np.array(dataframe[c]))
        mask =((dataframe < (mean - std_times*std)) | (dataframe > (mean+std_times*std)))
        dataframe.loc[mask[c],c] = np.nan
    
    dataframe = dataframe.interpolate()
    return dataframe

In [ ]:
####### Set up experiment parameters ###############

#Logging parameters
experiment_id = str(uuid.uuid4())
now = datetime.now()
date_time_string = now.strftime("%d-%m-%Y-%H-%M-%S")
# For potential tensorboard use
log_dir="logs/profile/" + date_time_string
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = 3)

#Training parameters
epochs = 120
batch_size = 64
starting_lr = 1e-3
optimizer = tfa.optimizers.AdamW(learning_rate=starting_lr, weight_decay=1e-4)
min_lr = 2e-6
loss ='mse'
    
    
log_name="logs/"+ F"{experiment_id}-{date_time_string}_train_history"
log_history_callback = CSVLogger(log_name)
filepath= experiment_id+"-weights.{epoch:02d}-{val_loss:.4f}.h5"
save_model_callback = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

# Unused callbacks after all
#lr_log_name = "logs/"+ F"{experiment_id}-{date_time_string}_learning_rate_history"
#lr_log_callback = LearningRateLogger(lr_log_name)
#lr_reducer_callback = ReduceLROnPlateau(factor=0.5,cooldown=0,patience=20,min_delta=0.001,min_lr=min_lr, verbose=1)
#lr_schedule = LearningRateScheduler(lambda epoch, lr: 1e-8 * 10**(epoch / 20))
#early_stopping_callback = EarlyStopping(monitor='val_loss', patience=40, min_delta=0.001)

callbacks_list = [log_history_callback, save_model_callback]#,lr_log_callback, lr_reducer_callback, early_stopping_callback]

#Dataset parameters
window_length = 192
forecast_horizon = 48
preprocessor = preprocessing.MinMaxScaler()
out_preprocessor = preprocessing.MinMaxScaler()
shuffle_train_set = True
scale_output = True
training_percentage = 0.9
experiment_target = F"Forecasting,{forecast_horizon} steps ahead"
experiment_complete = False

In [ ]:
############## Set up model ##########################
class MTCNAModel(tf.keras.Model):
    
    def __init__(self, tcn_layer_num,tcn_kernel_size,tcn_filter_num,window_size,forecast_horizon,num_output_time_series, use_bias, kernel_initializer, tcn_dropout_rate,tcn_dropout_format,tcn_activation, tcn_final_activation, tcn_final_stack_activation):
        super(MTCNAModel, self).__init__()


        self.num_output_time_series = num_output_time_series
        

        #Create stack of TCN layers    
        self.lower_tcn = TCNStack(tcn_layer_num,tcn_filter_num,tcn_kernel_size,window_size,use_bias,kernel_initializer,tcn_dropout_rate,tcn_dropout_format,tcn_activation,tcn_final_activation, tcn_final_stack_activation)
        
        self.downsample_att = DownsampleLayerWithAttention(num_output_time_series,window_size, tcn_kernel_size, forecast_horizon, kernel_initializer, None)
    
        
        
    def call(self, input_tensor):
        x = self.lower_tcn(input_tensor)
        x, distribution = self.downsample_att([x,input_tensor])
        return [x[:,i,:] for i in range(self.num_output_time_series)]#, distribution

In [ ]:
################ Prepare dataset ###########################

### Note details for logging purposes
dataset_description = "Burnett river sensor data"
dataset_preprocessing = """Drop TIMESTAMP, Replace outliers more than 3*std on input data with Nan,
pd.interpolate() for NaN values"""

# Read csv in pandas
data_files = []

for year in range(2014,2019):
    data_file = pd.read_csv(F"Datasets/burnett-river-trailer-quality-{year}.csv")
    data_files.append(data_file)
data = pd.concat(data_files,axis=0)

# Change type of temp to avoid errors
data = data.astype({'Temp_degC':'float64'})

#Create date object for easy splitting according to dates
dateobj = pd.to_datetime(data['TIMESTAMP'])

### For now remove timestamp and output valuesdata = remove_outliers_and_interpolate(data, std_times=3)
data = data.drop(columns=["TIMESTAMP","RECORD"],axis=1)
 
data = remove_outliers_and_interpolate(data, std_times=3)

In [ ]:
## Add date object for splitting
data['DateObj'] = dateobj

#Split data based on dates
training_start_date = pd.Timestamp(year=2014,month=3,day=1)

# Preceding values used only for creating final graph and predicting first values of test set
holdout_preceding_date = pd.Timestamp(year=2017, month=3, day=1)
holdout_set_start_date = pd.Timestamp(year=2017, month=4, day=1)
holdout_set_end_date = pd.Timestamp(year=2018, month=4, day=1)

training_data = data.loc[(data['DateObj']>=training_start_date) & (data['DateObj'] < holdout_set_start_date)]
test_data = data.loc[(data['DateObj'] >= holdout_set_start_date) & (data['DateObj'] < holdout_set_end_date)]
pre_evaluation_period = data.loc[(data['DateObj'] >= holdout_preceding_date) & (data['DateObj'] < holdout_set_start_date)]

## Keep iput variables
input_variables = ['Temp_degC', 'EC_uScm', 'pH', 'Turbidity_NTU', 'Chloraphylla_ugL']

training_data = training_data[input_variables]
test_data = test_data[input_variables]

In [ ]:
##Select prediction target
targets = ['Temp_degC', 'EC_uScm', 'pH', 'Turbidity_NTU','Chloraphylla_ugL']
labels = np.array(training_data[targets])

if scale_output:
    out_preprocessor.fit(labels)
    if "Normalizer" in str(out_preprocessor.__class__):
        ## Save norm so in case of normalizer we can scale the predictions correctly
        out_norm = np.linalg.norm(labels)
        labels = preprocessing.normalize(labels,axis=0)
    else:
        labels= out_preprocessor.transform(labels)


num_input_time_series = training_data.shape[1]


### Make sure data are np arrays in case we skip preprocessing
training_data = np.array(training_data)

# #### Fit preprocessor to training data
preprocessor.fit(training_data)

if "Normalizer" in str(preprocessor.__class__):
    ## Save norm so in case of normalizer we can scale the test_data correctly
    in_norm = np.linalg.norm(training_data,axis=0)
    training_data = preprocessing.normalize(training_data,axis=0)
else:
    training_data = preprocessor.transform(training_data)

In [ ]:

### Create windows for all data
data_windows = windowed_dataset(training_data[:-forecast_horizon],num_input_time_series,window_length)
label_windows = windowed_forecast(labels[window_length:],forecast_horizon)

### Transpose outputs to agree with model output
label_windows = np.transpose(label_windows,[0,2,1])


samples = data_windows.shape[0]


## Shuffle windows
if shuffle_train_set:
    data_windows, label_windows = shuffle_arrays_together(data_windows,label_windows)

### Create train and validation sets
train_x = data_windows
train_y = [label_windows[:,i,:] for i in range(len(targets))]


## In order to use all days of test set for prediction, append training window from preceding period
pre_test_train = pre_evaluation_period[test_data.columns][-window_length:]
test_data = pd.concat([pre_test_train,test_data])

## Create windowed test set with same process
test_labels = np.array(test_data[targets])

#### Preprocess data
test_data = np.array(test_data)

if "Normalizer" in str(preprocessor.__class__):
    test_data = test_data/in_norm
else:
    test_data = preprocessor.transform(test_data)

test_x = windowed_dataset(test_data[:-forecast_horizon],num_input_time_series,window_length)
test_y = np.transpose(windowed_forecast(test_labels[window_length:],forecast_horizon),[0,2,1])

## Create pre test period for visualization
pre_test_target = np.vstack((np.array(pre_evaluation_period[targets]),test_labels[:window_length]))

total_samples = train_x.shape[0] + test_x.shape[0]

In [ ]:
##################### Initialize model parameters ########################
## For simplicity all time series TCNs have the same parameters, though it is relatively easy to change this
tcn_kernel_size = 3
tcn_layer_num = 7
tcn_use_bias = True
tcn_filter_num = 64
tcn_kernel_initializer = 'random_normal'
tcn_dropout_rate = 0.5
tcn_dropout_format = "channel"
tcn_activation = 'relu'
tcn_final_activation = 'linear'
tcn_final_stack_activation = 'relu'

In [ ]:
# ### Check for GPU


## Make only given GPU visible   
gpus = tf.config.experimental.list_physical_devices('GPU')
mirrored_strategy = None

print("GPUs Available: ", gpus)
if len(gpus)==0:
    device = "CPU:0"
else:
    print("Enter number of gpus to use:")
    gpu_num = input()
    if len(gpu_num)!=0 and gpu_num.isdigit():
        gpu_num = int(gpu_num)
    if gpu_num==1:
        print("Enter index of GPU to use:")
        gpu_idx = input()
        if len(gpu_idx)!=0 and gpu_idx.isdigit():
            gpu_idx = int(gpu_idx)
        tf.config.experimental.set_visible_devices(gpus[gpu_idx], 'GPU')
        device = "GPU:0"
    else:
        mirrored_strategy = tf.distribute.MirroredStrategy(devices=[F"GPU:{i}" for i in range(gpu_num)])
        device = " ".join([F"GPU:{i}" for i in range(gpu_num)])

In [ ]:
# ##### Initialize model
loss = [loss]*len(targets)
if mirrored_strategy:
    with mirrored_strategy.scope():
        model = MTCNAModel(tcn_layer_num,tcn_kernel_size,tcn_filter_num,window_length,forecast_horizon,len(targets), tcn_use_bias, tcn_kernel_initializer, tcn_dropout_rate, tcn_dropout_format, tcn_activation, tcn_final_activation, tcn_final_stack_activation)
        model.compile(optimizer,loss,metrics=[tf.keras.metrics.RootMeanSquaredError()])
else:
    with tf.device(device):
        model = MTCNAModel(tcn_layer_num,tcn_kernel_size,tcn_filter_num,window_length,forecast_horizon,len(targets), tcn_use_bias, tcn_kernel_initializer, tcn_dropout_rate, tcn_dropout_format, tcn_activation, tcn_final_activation, tcn_final_stack_activation)
        model.compile(optimizer,loss,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
additional_experiment_notes = ""
log_experiment_details(just_print=True)
############## Note here any special notes about this experiment
print("Do you have any special notes for this experiment ? If yes, please write them here (or press Enter):")
additional_experiment_notes = input()
if len(additional_experiment_notes)==0:
    additional_experiment_notes = """No special notes"""

In [ ]:
#### Create log and start training

log_experiment_details()

start_time = tf.timestamp()

history = model.fit(x=train_x, y= train_y,validation_split=(1-training_percentage), batch_size=batch_size, epochs=epochs, callbacks=callbacks_list)

end_time = tf.timestamp()

duration = end_time - start_time

experiment_complete = True
log_experiment_details(complete=True)